## AI 
- Bu paket kalp hastalığı olup olmadığını ve ev fiyat tahmininde bulunan bir regresyon model üzerine gidilicektir.

- Model oluşturuken X ve y kavramları genel olarak X, giriş verileridir y, çıkış verileridir.

--- 

`california_house` verisini üst kısımda belirtilen parametrelere göre dataframe cevrime işlemi yapılmalıdır.

**Not:** index uyumu için `pd.Series` kullanılır. Kullanılmadan da atama yapılabilir. Direkt atama yapılırsa boyutların (satır sayısı) aynı olması lazımdır yoksa hata verir. `pd.Series` index eşleşmesi ile atama yapar index eşleşmediği kısımlara "NaN" değeleri ataması yapar.

---

---

In [8]:
import pickle
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split # Veriyi bölmek için kullanılır

from sklearn.ensemble import RandomForestClassifier # Sınıflandırma için kullanılır

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

clf = RandomForestClassifier()

# 1. adım "Getting the data ready"
heart_disease_data = pd.read_csv("data/heart-disease.csv") 
clf.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'sqrt',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'monotonic_cst': None,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

In [9]:
 # "target" kısmını ve axis=1 yani sütün dışında kısım ataması yapılır.
X = heart_disease_data.drop("target", axis=1)
y = heart_disease_data["target"]

# 2. adım "Choose the right estimator/algorithm for our problems"
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2) # Verileri bölmek için kullanılır (%20)
y.value_counts(), X_train.shape, X_test.shape, y_train.shape, y_test.shape

(target
 1    165
 0    138
 Name: count, dtype: int64,
 (242, 13),
 (61, 13),
 (242,),
 (61,))

---

Modeli eğitilicek verinin bölünmesinin nedeni model eğitimi sırasında karşılaşmadığı verilerle sınanması lazım bu sayede doğruluk oranı ölçülmüş gerçekci olur. Eğer eğitilen verilerle ölçüm yapılırsa 1 yani hepsi doğru sonucu çıkabilir.

---

In [10]:

clf.fit(X_train, y_train)  # Veri ile model eğitilir.

# 3. adım "Fit the model/algorithm and use it to make predictions on our data" (prediction)
# Belirtilen veride hasta olup olmadığını 1 - 0 olarak bir dizi verir
y_preds = clf.predict(X_test)  

# Belirtilen verinin yüzdelik olarak 1 mi 0 mı olduğu bu sekilde gösterilir
y_probs = clf.predict_proba(X_test)
score = clf.score(X_test, y_test) # Doğruluk oranını verir.
score2 = accuracy_score(y_test, y_preds)   # Doğruluk oranını verir.

# 4. Evaluating a model
print(f"(score) Model Coef: %{score*100:.2f}")
print(f"(score2) Model Coef: %{score2*100:.2f}")

(score) Model Coef: %81.97
(score2) Model Coef: %81.97


---
## confusion matrix
Çabraz şekilde (Sol üst kısımdan sağ alt kısam) şeklinde doğru tahmin sayısıdır.
- True Positive (TP): Modelin pozitif olarak doğru tahmin ettiği örneklerin sayısı.
- True Negative (TN): Modelin negatif olarak doğru tahmin ettiği örneklerin sayısı.
- False Positive (FP): Modelin pozitif olarak yanlış tahmin ettiği örneklerin sayısı (Type I error).
- False Negative (FN): Modelin negatif olarak yanlış tahmin ettiği örneklerin sayısı (Type II error).
---

In [11]:
print(classification_report(y_test, y_preds)) # Model hakkında bilgi verir.

# Modelin actual - predicted verir. ((TN, FP), (FN, TP))
#                   Tahmin Edilen
#                 Negatif    Pozitif
# Gerçek Negatif   TN         FP
# Gerçek Pozitif   FN         TP
confusion_matrix(y_test, y_preds)             

              precision    recall  f1-score   support

           0       0.81      0.71      0.76        24
           1       0.82      0.89      0.86        37

    accuracy                           0.82        61
   macro avg       0.82      0.80      0.81        61
weighted avg       0.82      0.82      0.82        61



array([[17,  7],
       [ 4, 33]])

In [12]:
# 5. Experiment to improve (hyperparameter tuning)
np.random.seed(42) # Random üreteçeği değerin her çalıştırmada aynı olmasını sağlar.
for i in range(10, 100, 20):  # Modelin "estimator" değerinde daha iyi sonuç verildiğine bakılır
    print(f"Trying model with {i} estimators...")
    clf = RandomForestClassifier(n_estimators=i).fit(X_train, y_train)
    print(f"Model accuracy on test set: {clf.score(X_test, y_test) * 100:.2f}%")
    print("")

Trying model with 10 estimators...
Model accuracy on test set: 77.05%

Trying model with 30 estimators...
Model accuracy on test set: 80.33%

Trying model with 50 estimators...
Model accuracy on test set: 80.33%

Trying model with 70 estimators...
Model accuracy on test set: 80.33%

Trying model with 90 estimators...
Model accuracy on test set: 80.33%



In [13]:
# 6. Load and save model
pickle.dump(clf, open("model/random_forst_model_1.pkl", "wb")) # Eğitilen modelin kaydedilmesini sağlar.
load_model = pickle.load(open("model/random_forst_model_1.pkl", "rb")) # Modelin yüklenmesini sağlar.
print(load_model.score(X_test, y_test))

0.8032786885245902
